In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%config InlineBackend.figure_format='retina'
plt.rcParams['figure.figsize'] = (8, 6)

In [45]:
class Experiment:
    def __init__(self, y_u_data_file_path, params):
        self.y_u_data = pd.read_csv(y_u_data_file_path)
        
        self.rho_kgpm3 = params['rho_kgpm3']
        self.v_inf_mps = params['v_inf_mps']
        self.y_loc_m = self.y_u_data['y'].values * 0.01
        self.u_mps = self.y_u_data['u'].values
        
        self.w_CV_m = params['w_CV_m']
        self.h_CV_m = self.y_loc_m[-1] - self.y_loc_m[0]
        self.A_CV_m2 = self.w_CV_m * self.h_CV_m
        
        self.q_inf_Pa = params['q_inf_Pa']
        self.dP_Pa = params['dP_Pa']
        self.d_obj_m = params['d_obj_m']
        self.A_obj_m2 = self.d_obj_m * self.w_CV_m
        
        # Drag calculation
        M1x = self.rho_kgpm3 * (self.v_inf_mps ** 2) * self.A_CV_m2
        M2x = self.w_CV_m * np.trapz(self.rho_kgpm3 * (self.u_mps ** 2), self.y_loc_m)
        M_term = M1x - M2x
        P_term = self.dP_Pa * self.A_CV_m2
        self.drag_N = M_term + P_term
        self.cd = self.drag_N / (self.q_inf_Pa * self.A_obj_m2)
        
    def wake_survey(self):
        plt.plot(self.u_mps, self.y_loc_m, 'o')
        plt.xlim(0, self.u_mps[-1] + 0.5)
        plt.grid(alpha=0.3)
        plt.title("Wake survey for HBR")
        plt.xlabel("u (m/s)")
        plt.ylabel("y-location (m)")
        
        plt.show()
        
    def report_drag(self):
        print("Drag: {} N".format(self.drag_N))
        print("cd: {}".format(self.cd))

In [51]:
HBR = Experiment(
    'data/y_u_hbr.csv',
    {
        'rho_kgpm3': 1.1598,
        'v_inf_mps': 10.56,
        'w_CV_m': 0.3,
        'q_inf_Pa': 64.55,
        'dP_Pa': 11.74,
        'd_obj_m': 0.04204
    }
)

HBR.report_drag()

Drag: 1.64752946124 N
cd: 2.023731915088061


In [52]:
LBR = Experiment(
    'data/y_u_lbr.csv',
    {
        'rho_kgpm3': 1.158,
        'v_inf_mps': 23.68,
        'w_CV_m': 0.3,
        'q_inf_Pa': 324.71,
        'dP_Pa': 30.32,
        'd_obj_m': 0.01842
    }
)

LBR.report_drag()

Drag: 3.8806749281099964 N
cd: 2.162722111864553
